In [1]:
import numpy as np

# Problem 1
def Newton_Cotes(g, a, b, N, method):
    i = 0
    sum = 0
    if method == 'midpoints':
        x = a + (b - a)/(2 * N)
        while i < N:
            sum += g(x) * (b - a)/N
            x += (b - a)/N
            i += 1
    elif method == 'trapezoid':
        x = a + (b - a)/N
        sum += g(a) * (b - a)/(2 * N)
        sum += g(b) * (b - a)/(2 * N)
        while i < N - 1:
            sum += g(x) * (b - a)/N
            x += (b - a)/N
            i += 1
    elif method == 'simpsons':
        x = a + (b - a)/(2 * N)
        sum += g(a) * (b - a)/(6 * (N + 1))
        sum += g(b) * (b - a)/(6 * (N + 1))
        while i < 2 * N - 2:
            if i % 2 == 0:
                sum += (4/(6 * (N + 1))) * g(x) * (b-a)
            else:
                sum += (2/(6 * (N + 1))) * g(x) * (b-a)
            x += (b - a)/(2 * N)
            i += 1
    return sum

In [2]:
g = lambda x: (0.1*x**4 - 1.5*x**3 + 0.53*x**2 + 2*x + 1)
a = -10
b = 10
N = 2000
print(Newton_Cotes(g,a,b,N,'midpoints'))
print(Newton_Cotes(g,a,b,N,'trapezoid'))
print(Newton_Cotes(g,a,b,N,'simpsons'))

4373.329911667478
4373.340176666212
4373.985091534279


In [3]:
# Problem 2
from scipy.stats import norm
def normal(mu, sigma, N, k):
    nodes = np.linspace(mu - k * sigma, mu + k * sigma, N)
    weights = np.zeros(N)
    weights[0] = norm.cdf((nodes[0] + nodes[1])/2, mu, sigma)
    weights[N-1] = 1- norm.cdf((nodes[N-2] + nodes[N-1])/2, mu, sigma)
    for i in range(1, N-1):
        weights[i] = norm.cdf((nodes[i + 1]+nodes[i])/2, mu, sigma) - norm.cdf((nodes[i]+nodes[i-1])/2, mu, sigma)
    return nodes, weights

print(normal(0, 1, 11, 3))
print(normal(0, 1, 11, 3)[1].sum())

(array([-3. , -2.4, -1.8, -1.2, -0.6,  0. ,  0.6,  1.2,  1.8,  2.4,  3. ]), array([ 0.00346697,  0.01439745,  0.04894278,  0.11725292,  0.19802845,
        0.23582284,  0.19802845,  0.11725292,  0.04894278,  0.01439745,
        0.00346697]))
1.0


In [4]:
# Problem 3
def log_normal(mu, sigma, N, k):
    nodes, weights = normal(mu, sigma, N, k)
    nodes = np.exp(nodes)
    return nodes, weights

In [5]:
# Problem 4
expected_income = np.dot(log_normal(10.5, 0.8, 200, 4)[0], log_normal(10.5, 0.8, 200, 4)[1])
print(expected_income)
mu, sigma = np.array([10.5, 0.8])
true_mean = np.exp(mu + sigma ** 2/2)
print(true_mean)

50007.1062252
50011.0870085


In [6]:
# Problem 5
import scipy.optimize as opt
import math

a = -10
b = 10

def eqs(x):
    eq1 = x[0] + x[1] + x[2] - (b - a)
    eq2 = x[0] * x[3] + x[1] * x[4] + x[2] * x[5] - ((1/2) * b ** 2 - (1/2) * a ** 2)
    eq3 = x[0] * x[3] ** 2 + x[1] * x[4] ** 2 + x[2] * x[5] ** 2 - ((1/3) * b ** 3 - (1/3) * a ** 3)
    eq4 = x[0] * x[3] ** 3 + x[1] * x[4] ** 3 + x[2] * x[5] ** 3 - ((1/4) * b ** 4 - (1/4) * a ** 4)
    eq5 = x[0] * x[3] ** 4 + x[1] * x[4] ** 4 + x[2] * x[5] ** 4 - ((1/5) * b ** 5 - (1/5) * a ** 5)
    eq6 = x[0] * x[3] ** 5 + x[1] * x[4] ** 5 + x[2] * x[5] ** 5 - ((1/6) * b ** 6 - (1/6) * a ** 6)
    return (eq1, eq2, eq3, eq4, eq5, eq6)
            
            
x = opt.root(eqs,(5, 10, 5, -6, 3, 6), tol = 1e-5).x
print(x)
weights = x[:3]
points = x[3: 6]

gaussian_quad = 0
for i in range(3): 
    gaussian_quad  += (g(points[i])) * (weights[i])
print(gaussian_quad)
                                        

[  5.55555255e+00   8.88889452e+00   5.55555293e+00  -7.74596774e+00
  -8.77787805e-06   7.74596763e+00]
4373.33301542


In [7]:
# Problem 6
from scipy.integrate import quad
print(quad(g, -10, 10)[0])

4373.333333333334


The result from scipy.integrate.quad is very close to the true value.

In [8]:
# Problem 7
def monte_carlo(N, omega = [-1, 1, -1, 1], g = lambda x, y: x ** 2 + y ** 2):
    sum = 0
    x = np.zeros(N)
    y = np.zeros(N)
    for i in range(N):
        x[i] = 2 * np.random.random_sample() - 1
        y[i] = 2 * np.random.random_sample() - 1
        if g(x[i], y[i]) <= 1:
            sum += 1
    return (omega[1]-omega[0]) * (omega[3] - omega[2]) * sum/N

monte_carlo(100000)

3.14048

In [20]:
# Problem 8
from math import sqrt, exp

def is_prime(x):
    if x == 2: return True
    if x < 2 or x % 2 == 0: return False
    return not any(x % i == 0 for i in range(3, int(sqrt(x)) + 1, 2))

def find_prime(n):
    """Return the nth prime number"""
    primes = []
    i = 1
    k = 1
    while i <= n :
        if is_prime(k + 1) == True:
            primes.append(k + 1)
            i += 1
        k += 1
    primes = np.asarray(primes)
    return primes

def prob8(n, d, t):
    x = np.zeros((n, d))
    primes = find_prime(d)
    if t == 'Weyl':
        for i in range(n):
            for j in range(d):
                k = i + 1
                x[i, j] = float(k * sqrt(primes[j])) - int(k * sqrt(primes[j]))
    if t == 'Haber':
        for i in range(n):
            for j in range(d):
                k = i + 1
                x[i, j] = float(k * (k + 1)/2 * sqrt(primes[j])) - int(k *(k + 1)/2 * sqrt(primes[j]))
    if t == 'Niederreiter':
        for i in range(n):
            for j in range(d):
                k = i + 1
                m = j + 1
                x[i, j] = float(k * 2 ** (m/(m + 1))) - int(k * 2 ** (m/(m + 1)))
    if t == 'Baker':
        for i in range(n):
            for j in range(d):
                k = i + 1
                x[i, j] = float(k * exp(primes[j])) - int(k * exp(primes[j]))
    return x[n - 1,:]

print(prob8(5, 2,'Weyl'))
print(prob8(5, 2,'Haber'))
print(prob8(5, 2,'Niederreiter'))
print(prob8(5, 2,'Baker'))


[ 0.07106781  0.66025404]
[ 0.21320344  0.98076211]
[ 0.07106781  0.93700526]
[ 0.94528049  0.42768462]


In [22]:
# Problem 9
def monte_carlo_quasi1(N, omega = [-1, 1, -1, 1], g = lambda x, y: x ** 2 + y ** 2):
    sum = 0
    A = np.zeros((N, 2))
    for i in range(N):
        A[i] = 2 * prob8(i+1, 2, 'Weyl') - 1
        if g(A[i, 0], A[i, 1]) <= 1:
            sum += 1
    return (omega[1]-omega[0]) * (omega[3] - omega[2]) * sum/N

def monte_carlo_quasi2(N, omega = [-1, 1, -1, 1], g = lambda x, y: x ** 2 + y ** 2):
    sum = 0
    A = np.zeros((N, 2))
    for i in range(N):
        A[i] = 2 * prob8(i+1, 2, 'Haber') - 1
        if g(A[i, 0], A[i, 1]) <= 1:
            sum += 1
    return (omega[1]-omega[0]) * (omega[3] - omega[2]) * sum/N

def monte_carlo_quasi3(N, omega = [-1, 1, -1, 1], g = lambda x, y: x ** 2 + y ** 2):
    sum = 0
    A = np.zeros((N, 2))
    for i in range(N):
        A[i] = 2 * prob8(i+1, 2, 'Niederreiter') - 1
        if g(A[i, 0], A[i, 1]) <= 1:
            sum += 1
    return (omega[1]-omega[0]) * (omega[3] - omega[2]) * sum/N

def monte_carlo_quasi4(N, omega = [-1, 1, -1, 1], g = lambda x, y: x ** 2 + y ** 2):
    sum = 0
    A = np.zeros((N, 2))
    for i in range(N):
        A[i] = 2 * prob8(i+1, 2, 'Baker') - 1
        if g(A[i, 0], A[i, 1]) <= 1:
            sum += 1
    return (omega[1]-omega[0]) * (omega[3] - omega[2]) * sum/N

print(monte_carlo_quasi1(10000))
print(monte_carlo_quasi2(10000))
print(monte_carlo_quasi3(10000))
print(monte_carlo_quasi4(10000))

3.142
3.146
3.1388
3.1432
